In [3]:
# ============================================================
# Weather Trend Forecasting: 02 Data Cleaning (Fixed & Improved)
# ============================================================

# ---------------------------
# Step 0: Setup project path
# ---------------------------
import sys
import os

# Add project root to sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)
print("Project root added to sys.path:", project_root)

# ---------------------------
# Step 1: Imports
# ---------------------------
import pandas as pd
import numpy as np
from scipy import stats
from IPython.display import display

# ---------------------------
# Step 2: Load raw dataset
# ---------------------------
raw_csv_path = os.path.join(project_root, "data", "raw", "GlobalWeatherRepository.csv")
if not os.path.exists(raw_csv_path):
    raise FileNotFoundError(f"Dataset not found at {raw_csv_path}.")

df = pd.read_csv(raw_csv_path)

# ---------------------------
# Step 2a: Convert date column
# ---------------------------
df["last_updated"] = pd.to_datetime(df["last_updated"], errors="coerce")

# Warn if invalid dates
num_invalid_dates = df["last_updated"].isna().sum()
if num_invalid_dates > 0:
    print(f"Warning: {num_invalid_dates} invalid 'last_updated' dates found.")

# ---------------------------
# Step 2b: Ensure essential columns
# ---------------------------
for col in ["location_name", "country"]:
    if col not in df.columns:
        df[col] = "Unknown"

# ---------------------------
# Step 3: Drop duplicates
# ---------------------------
df = df.drop_duplicates()
print(f"After dropping duplicates: {df.shape}")

# ---------------------------
# Step 4: Handle missing numeric values
# ---------------------------
numeric_cols = df.select_dtypes(include=np.number).columns.tolist()

# Fill missing numeric values by median per location + month (transform preserves index)
for col in numeric_cols:
    df[col] = df.groupby(['location_name', df['last_updated'].dt.month])[col] \
                 .transform(lambda x: x.fillna(x.median()))

# Optional: Smooth remaining missing values using rolling mean per location
for col in numeric_cols:
    df[col] = df.groupby('location_name')[col] \
                 .transform(lambda x: x.fillna(x.rolling(3, min_periods=1).mean()))

# ---------------------------
# Step 5: Handle missing categorical values
# ---------------------------
categorical_cols = df.select_dtypes(include='object').columns.tolist()
for col in categorical_cols:
    df[col] = df[col].fillna("Unknown")

# ---------------------------
# Step 6: Remove extreme outliers (z-score > 4)
# ---------------------------
for col in numeric_cols:
    z_scores = np.abs(stats.zscore(df[col], nan_policy='omit'))
    df.loc[z_scores > 4, col] = np.nan

# Interpolate any remaining missing numeric values linearly
df[numeric_cols] = df[numeric_cols].interpolate(method='linear', limit_direction='both')

# ---------------------------
# Step 7: Save cleaned dataset
# ---------------------------
processed_dir = os.path.join(project_root, "data", "processed")
os.makedirs(processed_dir, exist_ok=True)

cleaned_csv_path = os.path.join(processed_dir, "weather_cleaned.csv")
df.to_csv(cleaned_csv_path, index=False)
print(f"✅ Cleaned dataset saved to: {cleaned_csv_path}")

# ---------------------------
# Step 8: Display overview
# ---------------------------
print("\nCleaned Dataset Overview:")
print(f"Shape: {df.shape}")
display(df.head())


Project root added to sys.path: c:\Users\kumar\OneDrive\Desktop\Pm accelerator\weather-trend-forecast
After dropping duplicates: (97629, 41)
✅ Cleaned dataset saved to: c:\Users\kumar\OneDrive\Desktop\Pm accelerator\weather-trend-forecast\data\processed\weather_cleaned.csv

Cleaned Dataset Overview:
Shape: (97629, 41)


,country,location_name,latitude,longitude,timezone,last_updated_epoch,last_updated,temperature_celsius,temperature_fahrenheit,condition_text,...,air_quality_PM2.5,air_quality_PM10,air_quality_us-epa-index,air_quality_gb-defra-index,sunrise,sunset,moonrise,moonset,moon_phase,moon_illumination
0,Afghanistan,Kabul,34.52,69.18,Asia/Kabul,1.715849e+09,2024-05-16 13:15:00,26.6,79.8,Partly Cloudy,...,8.4,26.6,1.0,1.0,04:50 AM,06:50 PM,12:12 PM,01:11 AM,Waxing Gibbous,55.0
1,Albania,Tirana,41.33,19.82,Europe/Tirane,1.715849e+09,2024-05-16 10:45:00,19.0,66.2,Partly cloudy,...,1.1,2.0,1.0,1.0,05:21 AM,07:54 PM,12:58 PM,02:14 AM,Waxing Gibbous,55.0
2,Algeria,Algiers,36.76,3.05,Africa/Algiers,1.715849e+09,2024-05-16 09:45:00,23.0,73.4,Sunny,...,10.4,18.4,1.0,1.0,05:40 AM,07:50 PM,01:15 PM,02:14 AM,Waxing Gibbous,55.0
3,Andorra,Andorra La Vella,42.50,1.52,Europe/Andorra,1.715849e+09,2024-05-16 10:45:00,6.3,43.3,Light drizzle,...,0.7,0.9,1.0,1.0,06:31 AM,09:11 PM,02:12 PM,03:31 AM,Waxing Gibbous,55.0
4,Angola,Luanda,-8.84,13.23,Africa/Luanda,1.715849e+09,2024-05-16 09:45:00,26.0,78.8,Partly cloudy,...,183.4,262.3,5.0,10.0,06:12 AM,05:55 PM,01:17 PM,12:38 AM,Waxing Gibbous,55.0
